In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
experiment = "experiment-traffic-lights"

### Reading input data

In [4]:
with open(f'traffic/input_files/{experiment}/traffic.json', "r") as read_file:
    drivers_input_data = json.load(read_file)
with open(f'traffic/input_files/{experiment}/drivers.json', "r") as read_file:
    traffic_input_data = json.load(read_file)
with open(f'traffic/input_files/{experiment}/lights.json', "r") as read_file:
    lights_input_data = json.load(read_file)

### Reading output data

In [12]:
agent_data = pd.read_csv(f'traffic/output_files/{experiment}/agent_data.csv')
agent_data["Velocity"] = agent_data["Velocity"].apply(lambda x: float(x.split(" ")[0].replace("[","")))
agent_data.head()
agent_data[agent_data["AgentID"]==40]

,Step,AgentID,X,Y,Velocity,Current_lane,Is_alive
110337,14132,40,0.387640,100.0,0.433468,1,True
110347,14133,40,0.776280,100.0,0.433468,1,True
110357,14134,40,1.165920,100.0,0.433468,1,True
110367,14135,40,1.556560,100.0,0.433468,1,True
110377,14136,40,1.948200,100.0,0.433468,1,True
...,...,...,...,...,...,...,...
136065,16897,40,1197.898961,100.0,0.433468,1,True
136070,16898,40,1198.332429,100.0,0.433468,1,True
136075,16899,40,1198.765897,100.0,0.433468,1,True
136080,16900,40,1199.199365,100.0,0.433468,1,True


## Flow rate

#### Flow rate: The flow rate is the number of cars that pass a certain point per unit of time.

In [5]:
nodes = []
for node in lights_input_data:
    nodes.append(node["pos"])
print("The x positions of nodes:")
print(nodes)

The x positions of nodes:
[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200]


In [6]:
t_start = 800
t_end = 1800
measure_point_x = 600
accepted_dist_delta = 10

# A function to calculate flow_rates easy
def calc_flow_rate(t_start,t_end,measure_point_x,accepted_delta ):
    return agent_data[(agent_data["Is_alive"]==True)&(agent_data["Step"]>t_start)&(agent_data["Step"]<t_end)&(abs(agent_data["X"]-measure_point_x)<accepted_dist_delta)].drop_duplicates("AgentID").shape[0]


fr = calc_flow_rate( t_start = t_start,
                t_end = t_end,
                measure_point_x = measure_point_x,
                accepted_delta = accepted_dist_delta)

print("flow rate = ",fr)
print(f"Between {t_start} and {t_end} timestamps, there was {fr} cars at point {measure_point_x} with rounding to {accepted_dist_delta}")


flow rate =  6
Between 800 and 1800 timestamps, there was 6 cars at point 600 with rounding to 10


In [7]:
## Rolling average 

def flow_rate_running_avg(t_start, t_end, measure_point_x, accepted_dist_delta, window_size,running_step=100):
    flow_rates = []
    for t in range(t_start, t_end + 1-window_size,running_step):
        flow_rates.append(calc_flow_rate(t, t + window_size, measure_point_x, accepted_dist_delta))
    running_avg = sum(flow_rates) / len(flow_rates)
#     print(flow_rates) # check it out, A.
    return running_avg

flow_rate_running_avg(800,5800,600,1000,1000)

11.463414634146341

## Density
#### Density: Traffic density is the number of cars per unit length of the road at any instant. 

In [8]:
x_start = 500
x_end = 600
time = 2000
def calc_density(x_start,x_end,time):
    return agent_data[(agent_data["Is_alive"]==True)&(agent_data["X"]>x_start)&(agent_data["X"]<x_end)&(agent_data["Step"]==time)].drop_duplicates("AgentID").shape[0]/(x_end-x_start)
d = calc_density(x_start,x_end,time)

# density at time 2000 between  two given nodes (their x_positions)
print(f"The number of cars between pos_x {x_start} and {x_end}, at time {time}, divided by the distance of {x_end-x_start} is equal {d}")

The number of cars between pos_x 500 and 600, at time 2000, divided by the distance of 100 is equal 0.04


In [9]:
# rolling average

def density_running_avg(x_start,x_end,t_start, t_end, running_step=5):
    densities = []
    for t in range(t_start, t_end +1, running_step):
        densities.append(calc_density(x_start,x_end,t))
    running_avg = sum(densities) / len(densities)
#     print(densities)
    return running_avg

#running average for the same two nodes, using time range, not specific timestamp this time, the result should be simmilar
density_running_avg(500,600,1800,2400)

0.026446280991735505

In [10]:
x_start = 500
x_end = 600
time = 2000
agent_data

,Step,AgentID,X,Y,Velocity,Current_lane,Is_alive
0,1,0,0.415049,166.666667,0.593567,2,True
1,1,1,0.007917,33.333333,0.408623,0,True
2,1,2,0.399195,100.000000,0.548394,1,True
3,1,3,0.000000,100.000000,0.640693,1,False
4,1,4,0.000000,166.666667,0.332989,2,False
...,...,...,...,...,...,...,...
239331,7346,43,1198.736299,100.000000,0.303324,1,True
239332,7347,43,1199.039623,100.000000,0.303324,1,True
239333,7348,43,1199.342947,100.000000,0.303324,1,True
239334,7349,43,1199.646270,100.000000,0.303324,1,True
